In [1]:
test = '''
contract sum(n : int){

    let x: int = 0;

    entry sum(n: int){
        let a: int = 2;
        let b: int = 3;
        while (a < b) {
            x = a + b;
            a = 5;
        }
        
        
    }
}
'''



In [2]:
from lexer.sly_lexer import TzScriptLexer, process_lexer_tokens 
from lexer.lex_token import Token
from parser.tzscript_grammar import TZSCRIPT_GRAMMAR,idx, num, typex, contract, ifx, elsex,equal, plus, minus, star, div,semi, colon, comma, dot, opar, cpar, ocur, ccur,let, func,entry,lessthanequal,greaterthanequal, iniquelaty, lessthan,greaterthan,equalequal, returnx
from parser.slr_parser import SLR1Parser, build_slr_ast

lexer = TzScriptLexer()
lexer_tokens = list(lexer.tokenize(test))
tokens = process_lexer_tokens(lexer_tokens)

parser = SLR1Parser(TZSCRIPT_GRAMMAR, verbose=True)

terminals = [token.token_type for token in tokens]
derivation = parser(terminals, True)

0 	 S' -> .<program>, 
	 <program> -> .contractid(<param-list>){<stat_list>},  

1 	 S' -> <program>.,  

2 	 <program> -> contract.id(<param-list>){<stat_list>},  

3 	 <program> -> contractid.(<param-list>){<stat_list>},  

4 	 <param> -> .id:type, 
	 <param-list> -> .<param>,<param-list>, 
	 <param-list> -> .<param>, 
	 <program> -> contractid(.<param-list>){<stat_list>},  

5 	 <param-list> -> <param>.,<param-list>, 
	 <param-list> -> <param>.,  

6 	 <param> -> .id:type, 
	 <param-list> -> <param>,.<param-list>, 
	 <param-list> -> .<param>,<param-list>, 
	 <param-list> -> .<param>,  

7 	 <param-list> -> <param>,<param-list>.,  

8 	 <param> -> id.:type,  

9 	 <param> -> id:.type,  

10 	 <param> -> id:type.,  

11 	 <program> -> contractid(<param-list>.){<stat_list>},  

12 	 <program> -> contractid(<param-list>).{<stat_list>},  

13 	 <program> -> contractid(<param-list>){.<stat_list>}, 
	 <return-stat> -> .return<expr>;, 
	 <def-entry> -> .entryid(<param-list>){<stat_list>}, 


In [3]:
productions, operations = derivation
ast = build_slr_ast(productions, operations, tokens)
from visitors.string_rep_visitor import FormatVisitor
format = FormatVisitor()
print(format.visit(ast))


\__ProgramNode: contract node.idx(n : int) [<stat>; ... <stat>;]
	\__AttrDeclarationNode: n : int
		\__VarDeclarationNode: let x = <expr> : int
			\__ ConstantNumNode: 0
		\__EntryDeclarationNode: Entry sum(n : int)
			\__AttrDeclarationNode: n : int
				\__VarDeclarationNode: let a = <expr> : int
					\__ ConstantNumNode: 2
				\__VarDeclarationNode: let b = <expr> : int
					\__ ConstantNumNode: 3
				\__WhileNode: while <expr> then [<stat>; ... <stat>;]
					\__ LessThanNode
						\__ VariableNode: a
						\__ VariableNode: b
					\__VarCallNode: x = <expr>
						\__ PlusNode
							\__ VariableNode: a
							\__ VariableNode: b
					\__VarCallNode: a = <expr>
						\__ ConstantNumNode: 5


In [4]:
from visitors.high_level_ir_generator_visitor import TzScriptToHighLevelIrVisitor
high_level_ir = TzScriptToHighLevelIrVisitor()
ir = high_level_ir.visit(ast)

mirame
mirame
statements in entry:  <parser.tzscript_ast.VarDeclarationNode object at 0x7ff69bbdf370>
statements in entry:  <parser.tzscript_ast.VarDeclarationNode object at 0x7ff69bbdf4f0>
statements in entry:  <parser.tzscript_ast.WhileNode object at 0x7ff69bbdfd60>
statement in while
here
nodos donde suma
here
nodos donde suma


In [5]:
from visitors.hl_string_repre import HLReprVisitor
hl_repr = HLReprVisitor()
print(hl_repr.visit(ir))

los:  [<intermediate_ast.high_level_ir_ast.IfEntryNode object at 0x7ff69b746d90>]
mmm [<intermediate_ast.high_level_ir_ast.PushValueNode object at 0x7ff69b7b65b0>, <intermediate_ast.high_level_ir_ast.PushVariableNode object at 0x7ff69b746b20>, <intermediate_ast.high_level_ir_ast.PushValueNode object at 0x7ff69b7b6790>, <intermediate_ast.high_level_ir_ast.PushVariableNode object at 0x7ff69b7b6760>, <intermediate_ast.high_level_ir_ast.WhileDeclarationNode object at 0x7ff69b7b6a60>]
operation node:  <intermediate_ast.high_level_ir_ast.LessThanNode object at 0x7ff69b7b6b80>
operation node:  <intermediate_ast.high_level_ir_ast.PlusNode object at 0x7ff69b7b6f10>
contract { entrypoint{...} storage{...} code{...}} 
	entrypoint  { <id,params> , <id,params> , ... , <id,params> }
		\__EntryPointDeclarationNode: sum ([<parser.tzscript_ast.AttrDeclarationNode object at 0x7ff69bbdf430>])
	storage { <id,type> , <id,type> , ... , <id,type> }
		\__StorageDeclarationNode: x : int
	code { <stat> , <stat>

In [6]:
from visitors.michelson_generator_visitor import MichelsonGenerator
visit_generator = MichelsonGenerator()
code = visit_generator.visit(ir)

reference counter:  {'a': 3, 'b': 2, 'x': 1}
push value node:  <class 'str'> <class 'str'>
push value node:  <class 'str'> <class 'str'>
estado de counnter {'a': 3, 'b': 2, 'x': 1}
estado de counnter {'a': 2, 'b': 2, 'x': 1}
code parameter int %sum;
storage int;
code {
UNPAIR;
PUSH nat 2;
PUSH nat 3;
DIG 1;
DIG 1;
DUP;
DIG 2;
SWAP;

first 3 num
second 2 num
estado de counnter {'a': 2, 'b': 1, 'x': 1}
estado de counnter {'a': 1, 'b': 1, 'x': 1}
code parameter int %sum;
storage int;
code {
UNPAIR;
PUSH nat 2;
PUSH nat 3;
DIG 1;
DIG 1;
DUP;
DIG 2;
SWAP;
LT;
LOOP {DIG 2;

first 3 num
second 0 int
current stack,  [<visitors.michelson_generator_visitor.StackValue object at 0x7ff69b79bdc0>, <visitors.michelson_generator_visitor.StackValue object at 0x7ff69b79b820>]
current-code:  parameter int %sum;
storage int;
code {
UNPAIR;
PUSH nat 2;
PUSH nat 3;
DIG 1;
DIG 1;
DUP;
DIG 2;
SWAP;
LT;
LOOP {DIG 2;
ADD;
DROP;

push value node:  <class 'str'> <class 'str'>
current stack,  [<visitors.michelson_